# Environment

In [1]:
!nvidia-smi

Sun May 23 07:08:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install libararies

In [2]:
!pip install timm
!pip install transformers
!pip install albumentations
!pip install git+https://github.com/ildoonet/pytorch-randaugment
!pip install git+https://github.com/ildoonet/pytorch-randaugment

  Cloning https://github.com/ildoonet/pytorch-randaugment to /tmp/pip-req-build-r92r8990
  Running command git clone -q https://github.com/ildoonet/pytorch-randaugment /tmp/pip-req-build-r92r8990
  Created wheel for RandAugment: filename=RandAugment-0.1-cp37-none-any.whl size=24212 sha256=bd8f946ac34b47c693930cb5ec9ccc08cf89d6bf5558cc5a4125e97930f2181b
  Stored in directory: /tmp/pip-ephem-wheel-cache-j0h2gqo5/wheels/0d/0e/e9/f8b70c1e233491338d524d867a7e959d10bb14a16bd5379b09
Successfully built RandAugment
  Cloning https://github.com/ildoonet/pytorch-randaugment to /tmp/pip-req-build-bgxag6mm
  Running command git clone -q https://github.com/ildoonet/pytorch-randaugment /tmp/pip-req-build-bgxag6mm
  Created wheel for RandAugment: filename=RandAugment-0.1-cp37-none-any.whl size=24212 sha256=aee0e8ffcfa0d1970288237921b9cba32c470aa71a5c5c2b1599193da2f15778
  Stored in directory: /tmp/pip-ephem-wheel-cache-3gwc443m/wheels/0d/0e/e9/f8b70c1e233491338d524d867a7e959d10bb14a16bd5379b09
Success

# Extract Datasets

In [ ]:
# !unzip /content/drive/MyDrive/dev/test.zip -d /content/drive/MyDrive/dev/test
# !unzip /content/drive/MyDrive/dev/train.zip -d /content/drive/MyDrive/dev/train

# Image Augmentation(HorizontalFlip)
#### - crop, rotation, randaugment 등의 Augmentation은 성능 향상이 없어 제거

In [3]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms, datasets
from torchvision.transforms import Resize, ToTensor, Normalize
from RandAugment import RandAugment

# transforms.RandomHorizontalFlip(1),
# transforms.CenterCrop((450,225)),
# transforms.RandomRotation(5),
# transforms.RandomHorizontalFlip(0),
# transforms.RandomHorizontalFlip(0),

train_transform = transforms.Compose([
                                    transforms.Resize((224, 224)),
                                    #transforms.RandomResizedCrop(190),
                                    transforms.RandomHorizontalFlip(0.5),
                                    #RandAugment(1,5),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                ])

test_transform = transforms.Compose([
                                    transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                ])

# Data Loader(train, val, test)를 위한 Custom Dataset Class 정의

In [4]:
def get_label(image_path):    #image 경로로부터 label을 추출
    if "dog" in image_path:
        return 0
    elif "elephant" in image_path:
        return 1
    elif "giraffe" in image_path:
        return 2
    elif "guitar" in image_path:
        return 3
    elif "horse" in image_path:
        return 4
    elif "house" in image_path:
        return 5
    elif "person" in image_path:
        return 6
    else:
        print(f"error!!! : {image_path}")
        return -1

class Dataset_Train(Dataset):    #Train Dataset 생성을 위한 Class. image와 label을 return함
    def __init__(self, data_list, transform):
        self.data_list = data_list
        self.transform = transform

    def __getitem__(self, idx):
        image_path = self.data_list[idx]
        label = get_label(image_path)
        img = Image.open(image_path)
        img = self.transform(img)
        return img, label
 
    def __len__(self):
        return len(self.data_list)

class Dataset_Test(Dataset):    #Test Dataset 생성을 위한 Class. image만 transform 하여 return
    def __init__(self, data_list, transform):
        self.data_list = data_list
        self.transform = transform

    def __getitem__(self, idx):
        image_path = self.data_list[idx]
        img = Image.open(image_path)
        img = self.transform(img)
        return img
 
    def __len__(self):
        return len(self.data_list)

# Base model, Hyperparameter 정의 및 시드 고정
#### - 앙상블에 사용한 model : hrnet_w40(stratified k-fold, train all datasets), hrnet_w64(train all datasets)
#### - 시도하였으나 성능이 나빠 사용하지 않은 모델 : efficientnet-b0, b3, b6
#### Hyperparameters
 - batch_size : 64
 - lr : 0.0001
 - momentum : 0.9
 - weight_decay :0.005
 - restarts(scheduler) : 5
 - num_epochs : 70

In [5]:
from torch import optim
from torch.optim import lr_scheduler
import torch
import numpy as np

random_seed = 11
num_classes = 7 
base_model = 'hrnet_w40'
batch_size = 64
lr = 0.0001
momentum = 0.9
weight_decay = 0.005
restarts = 5
num_epochs = 70
model_name = 'hrnet_w40'

torch.backends.cudnn.deterministic = True    #GPU 메모리 관리를 위해 작성
torch.backends.cudnn.benchmark = False
torch.manual_seed(random_seed)
np.random.seed(random_seed)

# 5 fold Division
#### - Startified k-fold를 위해 전체 데이터셋의 class 비율에 맞춰 fold를 5개로 나누어줌. 또한 전체 데이터셋도 학습한 모델을 가지기 위해 전체 데이터셋도 생성
#### - 클래스 데이터셋을 5등분하여 5등분점을 기준으로 각 fold(0~4)에 삽입

In [7]:
from glob import glob
from pprint import pprint
from collections import defaultdict
all_train_images = sorted(glob('/content/drive/MyDrive/dev/train/train/*/*'))
class_images = []
fold_images = [[] for _ in range(5)]
label_to_class = {0:'dog',1:'elephant',2:'giraffe',3:'guitar',4:'horse',5:'house',6:'person'}

for i in range(7):
    temp_arr = []
    class_image = sorted(glob(f'/content/drive/MyDrive/dev/train/train/{label_to_class[i]}/*'))
    class_images.append(class_image)

train_images = []
test_images = sorted(glob('/content/drive/MyDrive/dev/test/test/*/*'))
class_dict = defaultdict(int)

for elem in all_train_images:
    label = get_label(elem)
    class_dict[label]+=1

pprint(class_dict)

folds = 5

for k in class_dict.keys():    #각 클래스 데이터셋을 5등분하여 등분점의 index를 잡고, 해당 index를 기준으로 데이터를 나누어 fold마다 삽입
    fold_idx = [class_dict[k]//5,class_dict[k]*2//5,class_dict[k]*3//5,class_dict[k]*4//5,class_dict[k]]
    start = 0
    for i,elem in enumerate(fold_idx):
        end = fold_idx[i]
        fold_images[i].extend(class_images[k][start:end])
        start = end

train_folds = [[] for _ in range(5)]
val_folds = [[] for _ in range(5)]

for i in range(5):
    for j in range(5):
        if i==j:
            continue
        train_folds[i].extend(fold_images[j])
    print(f'{i} fold train data : {len(train_folds[i])}')
    val_folds[i] = fold_images[i]
    print(f'{i} fold val data : {len(val_folds[i])}')

defaultdict(<class 'int'>,
            {0: 329,
             1: 205,
             2: 235,
             3: 134,
             4: 151,
             5: 245,
             6: 399})
0 fold train data : 1361
0 fold val data : 337
1 fold train data : 1358
1 fold val data : 340
2 fold train data : 1358
2 fold val data : 340
3 fold train data : 1358
3 fold val data : 340
4 fold train data : 1357
4 fold val data : 341


# Initialize Loader
#### Dataset으로부터 loader를 만들어 줌(k-fold loader, all dataset loader, test loader)

In [8]:
train_datasets = [[] for _ in range(5)]
val_datasets = [[] for _ in range(5)]
train_loaders = [[] for _ in range(5)]
val_loaders = [[] for _ in range(5)]

for i in range(5):
    train_datasets[i] = Dataset_Train(train_folds[i],train_transform)
    val_datasets[i] = Dataset_Train(val_folds[i],test_transform)
    train_loaders[i] = DataLoader(train_datasets[i],batch_size=batch_size,shuffle=True,num_workers=2)
    val_loaders[i] = DataLoader(val_datasets[i],shuffle=False)

dataset_train_all = Dataset_Train(all_train_images,train_transform)
dataset_test = Dataset_Test(test_images,test_transform)

all_train_loader = DataLoader(dataset_train_all,batch_size=batch_size,shuffle=True,num_workers=2)
test_loader = DataLoader(dataset_test,shuffle=False)

for elem in all_train_images:    #label이 잘못되었는지 check
    if get_label(elem) not in [0,1,2,3,4,5,6]:
        print(elem)

# Create Model, Optimizer, Scheduler
#### - loss : CE loss 사용. focal loss는 성능 향상이 없어 제거
#### - optimizer : SGD (Adam 보다 실험 결과 성능이 좋아 사용)
#### - scheduler : CosineAnnealingWarmRestarts
#### - model : timm(torch image models) open source libarary를 이용해 생성


In [9]:
import torch
from torch import optim
from torch.optim import lr_scheduler
import timm
m = timm.create_model(base_model, pretrained=True, num_classes = num_classes)

optimizer = optim.SGD(m.parameters(),lr=lr,momentum=momentum,weight_decay=weight_decay)
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,restarts)
loss = torch.nn.CrossEntropyLoss()

# Train
#### - train 은 epoch 당 train->validation 순서로 이루어지며, validation accuracy가 최고를 갱신하면 best 모델로 저장하며 학습
#### - 해당 방식으로 0~4fold 및 데이터셋 전체를 학습

In [ ]:
from tqdm import tqdm
import time
import copy
import torch
device = torch.device('cuda')
model_save_dir = "/content/drive/MyDrive/dev/models"
def train(model,model_name,num_epochs,fold,criterion,optimizer,scheduler):
    model.to(device)
    train_loader = train_loaders[fold]
    val_loader = val_loaders[fold]
    best_eval_acc = 0.0
    best_eval_loss = 1.0
    for epoch in range(num_epochs):
        start = time.time()
        print(f'{epoch+1}/{num_epochs} epochs')
        print('-----------')
        train_loss, train_corrects, train_labels = 0,0,0
        eval_loss, eval_corrects, eval_labels = 0,0,0

        #train
        model.train()
        for inputs, labels in train_loader:   
            labels = labels.to(device)
            optimizer.zero_grad()
            inputs = inputs.to(device)
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            _, preds = torch.max(outputs,1)
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.item() * inputs.size(0)
            train_corrects += torch.sum(preds==labels.data)
            train_labels += len(labels)

        train_avg_loss = float(train_loss/train_labels)
        train_acc = float(train_corrects.double()/train_labels)

        print(f"train loss : {round(train_avg_loss,5)}")
        print(f"train acc : {round(train_acc,5)}")

        #val
        with torch.no_grad():
            model.eval()
            for inputs, labels in val_loader:   
                labels = labels.to(device)
                optimizer.zero_grad()
                inputs = inputs.to(device)
                outputs = model(inputs)
                loss = torch.tensor(0.0).to(device)
                

                _, preds = torch.max(outputs,1)            
                loss = criterion(outputs,labels)
                eval_loss += loss.item() * inputs.size(0)
                eval_corrects += torch.sum(preds==labels.data)
                eval_labels += len(labels)

                 
            eval_avg_loss = float(eval_loss/eval_labels)
            eval_acc = float(eval_corrects.double()/eval_labels)
            
            print(f"eval loss : {round(eval_avg_loss,5)}")
            print(f"eval acc : {round(eval_acc,5)}")

            if eval_acc>=best_eval_acc:    #validation acc 기준으로 모델 저장
                best_eval_acc = eval_acc
                torch.save(model.state_dict(),f'{model_save_dir}/{model_name}_{fold}fold_best.pth')
                print(f"best model saved : '{model_name}_{fold}fold_best.pt")
        time_spent = time.time() - start
        print(f"epoch train time : {round(time_spent//60)}min {round(time_spent%60)}sec")
    del loss
    del labels
    del inputs
    del outputs
    torch.cuda.empty_cache()
    torch.save(model.state_dict(),f'{model_save_dir}/{model_name}_ep_{epoch}.pth')

fold = 4
train(model=m, model_name=model_name, num_epochs=num_epochs, fold = fold, criterion=loss, optimizer=optimizer, scheduler=scheduler)

1/70 epochs
-----------
train loss : 1.8516
train acc : 0.23803
eval loss : 1.81326
eval acc : 0.23754
best model saved : 'hrnet_w40_4fold_best.pt
epoch train time : 0min 37sec
2/70 epochs
-----------
train loss : 1.75538
train acc : 0.27119
eval loss : 1.73616
eval acc : 0.28152
best model saved : 'hrnet_w40_4fold_best.pt
epoch train time : 0min 37sec
3/70 epochs
-----------
train loss : 1.67886
train acc : 0.35225
eval loss : 1.67354
eval acc : 0.3695
best model saved : 'hrnet_w40_4fold_best.pt
epoch train time : 0min 37sec
4/70 epochs
-----------
train loss : 1.6088
train acc : 0.45615
eval loss : 1.61296
eval acc : 0.44282
best model saved : 'hrnet_w40_4fold_best.pt
epoch train time : 0min 37sec
5/70 epochs
-----------
train loss : 1.53918
train acc : 0.53279
eval loss : 1.55204
eval acc : 0.5044
best model saved : 'hrnet_w40_4fold_best.pt
epoch train time : 0min 37sec
6/70 epochs
-----------
train loss : 1.46454
train acc : 0.59985
eval loss : 1.49157
eval acc : 0.55425
best model

# Inference
#### - 해당 fold의 best checkpoint를 불러와 inference
#### - soft vote ensemblem을 위해 argmax 하기 전의 logit 값을 pickle로 저장

In [ ]:
import pickle
import pandas as pd
load_epoch = num_epochs

cp_path = f'{model_save_dir}/{model_name}_{fold}fold_best.pth'
submission = pd.read_csv('/content/drive/MyDrive/dev/test_answer_sample_.csv')

all_predictions = []
all_predictions_logit = []
m.load_state_dict(torch.load(cp_path))
m.to(device)
m.eval()

for images in tqdm(test_loader):
    with torch.no_grad():
        images = images.to(device)
        pred = m(images)
        all_predictions_logit.append(pred)
        pred = pred.argmax()
        all_predictions.append(pred)
        
# save
with open(f'{model_save_dir}/{model_name}_fold{fold}.pickle', 'wb') as f:
    pickle.dump(all_predictions_logit, f, pickle.HIGHEST_PROTOCOL)
del m

100%|██████████| 350/350 [00:14<00:00, 23.98it/s]


# Export data to CSV
#### - 단일 모델 CSV 파일 생성

In [ ]:
submission['answer value'] = list(map(int,all_predictions))
submission.columns
submission = submission.drop(['Unnamed: 0'],axis=1)
submission.to_csv(f'/content/drive/MyDrive/dev/submission/{model_name}.csv')

In [ ]:
submission

,answer value
0,1
1,3
2,1
3,4
4,3
...,...
345,6
346,3
347,3
348,5


# Ensemble

#### - 이전에 저장했던 logit값을 불러와 weghted sum -> argmax를 구하는 형태로 soft voting
#### - soft vote 하되, Public Leaderboard 점수에 따라 weight를 차등 부여하여 Ensemble함
#### - 다양한 조합으로 앙상블 시도

In [69]:
ensembles = ['/content/drive/MyDrive/dev/models/hrnet_w40_hf_64_50ep_ep_50.pickle',
             '/content/drive/MyDrive/dev/models/hrnet_w40_hf_ep_70.pickle',
             '/content/drive/MyDrive/dev/models/hrnet_w40_ep_70_2.pickle',
             '/content/drive/MyDrive/dev/models/hrnet_w64_hf_ep_50.pickle',
             '/content/drive/MyDrive/dev/models/hrnet_w40_fold0.pickle',
             '/content/drive/MyDrive/dev/models/hrnet_w40_fold1.pickle',
             '/content/drive/MyDrive/dev/models/hrnet_w40_fold2.pickle',
             '/content/drive/MyDrive/dev/models/hrnet_w40_fold3.pickle',
             '/content/drive/MyDrive/dev/models/hrnet_w40_fold4.pickle',
             ]
             
weight = [1,4,1,1,1,1,1,1,1]
pred_final = []
all_data = []
for i,elem in enumerate(ensembles):
    with open(elem, 'rb') as f:
        data = pickle.load(f)
        all_data.append(data)

for i in range(len(data)):    #350 
    temp = all_data[0][i]*weight[0]
    for j in range(1,len(ensembles)):
        temp+=all_data[j][i]*weight[j]
    pred_final.append(temp.argmax())

submission = pd.read_csv('/content/drive/MyDrive/dev/test_answer_sample_.csv')
submission['answer value'] = list(map(int,pred_final))
submission = submission.drop(['Unnamed: 0'],axis=1)
submission.to_csv(f'/content/drive/MyDrive/dev/submission/submission_final.csv')